
# Data Cleaning

In [2]:
#!pip install pandas
#!pip install numpy
#!pip install nltk
#!pip install multiprocess
!pip install emoji
!pip install pytables
!pip install tables

# Import statements
import zipfile
import os

import shutil

import pandas as pd
import numpy as np 
import csv
import re
import string
import time
from datetime import datetime, timedelta, date


import multiprocess
from multiprocess import Pool, Process
import multiprocessing as mp



import emoji
import nltk
from nltk import pos_tag, pos_tag_sents
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
# from sklearn.feature_extraction.text import TfidfVectorizer

import warnings
warnings.filterwarnings("ignore")

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
stop_words = set(stopwords.words('english')) - set(['at', 'do', 'your', 'from', 'to', 'out', 'no', 'the'])

ERROR: Could not find a version that satisfies the requirement pytables
ERROR: No matching distribution found for pytables


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/priyanshkedia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/priyanshkedia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/priyanshkedia/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/priyanshkedia/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [52]:
# Extract zip file

def extract_zips(filename, temp_folder, FOLDER_NAME):
#   if os.path.isdir(temp_folder):
#     return FOLDER_NAME
  zip_ref = zipfile.ZipFile(filename, 'r')
  zip_ref.extractall(temp_folder)
  zip_ref.close()
  if os.path.isdir(os.path.join(TEMP_FOLDER, FOLDER_NAME)):
    pass
  else:
    FOLDER_NAME = ""
  return FOLDER_NAME

In [4]:
def create_filename(FILENAME, TEMP_FOLDER):
  FILES_NAMES = FILENAME.split(".")[0].split("/")[-1]
  FILES_NAMES = FILES_NAMES.rsplit("-", 1)[0]
  # FILES_NAMES = os.path.join(TEMP_FOLDER, FILES_NAMES)
  return FILES_NAMES

In [5]:
def get_date_string(FILESNAMES):
  date_string = FILESNAMES.split("-")[-1]
  return date_string

def strip_date(FILESNAMES):
  date_string = get_date_string(FILESNAMES)
  ob = datetime.strptime(date_string, "%Y%m%d")
  date_to_analyse = ob.strftime("%a %b %d")
  year = ob.strftime("%Y")
  return date_to_analyse, year

def get_date_numbers(date_string):
    ob = datetime.strptime(date_string, "%a %b %d")
    return ob.strftime("%d"), ob.strftime("%m")
    
def get_next_date(current_date,year):
    day, month = get_date_numbers(current_date)
    date = datetime(int(year), int(month), int(day))
    date += timedelta(days = 1)
    return date.strftime("%Y%m%d")
    
def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)
        
# h01-20201001-20201008
def get_date_range(FILESNAMES):
    filesnames = FILESNAMES.split('.')[0]
    date_string = filesnames.split("-")[-2:]
    start_date = strip_date(date_string[0])
    end_date = strip_date(date_string[1])
    start_date, start_year, end_date, end_year = start_date[0], start_date[1], end_date[0], end_date[1]
    start_day, start_month = get_date_numbers(start_date)
    end_day, end_month = get_date_numbers(end_date)
    
    start_date = date(int(start_year), int(start_month), int(start_day))
    end_date = date(int(end_year), int(end_month), int(end_day))
    return start_date, end_date 

In [6]:
def create_directory(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

In [7]:
def get_folder_name_from_zip(FILENAME):
    name = FILENAME.split('.')[0].split("/")[-1]
    return name
    

In [8]:
added_ids = {}

In [9]:
# Keep data in english only
def remove_other_langs(data):
  data = data[data['lang'] == 'en'].reset_index(drop=True)
  return data

In [10]:
def save_other_date(data, old_data, current_date, TEMP_FOLDER, FILES_NAMES, FOLDER_NAME):
    if data.equals(old_data):
        return
    date_to_save = get_next_date(current_date, year)
    print(date_to_save)
    #directory = "data"
    
    #directory = "/media/manikya_varshney/Backup/Yale Data/data"
    directory = '{}/{}'.format(TEMP_FOLDER, FOLDER_NAME)
    
    create_directory(directory)
    combined = pd.concat([data, old_data]).drop_duplicates(keep=False)
    
    combined.to_csv("{}/h01-{}.csv".format(directory, date_to_save))

In [11]:
# Keep specific date
def remove_other_dates(data, date_to_analyse, TEMP_FOLDER, FILES_NAMES, FOLDER_NAME):
  new_data = data[data['created_at'].str[:10] == date_to_analyse].reset_index(drop=True)
  save_other_date(new_data, data, date_to_analyse, TEMP_FOLDER, FILES_NAMES, FOLDER_NAME)
  return new_data

In [12]:
# Creating the RT Column
def create_rt_column(data):
  data['RT'] = data['text'].str[:2]=='RT'
  return data

In [13]:
# Parse CSV data
def parse_data_from_file(filename, date_to_analyse, TEMP_FOLDER, FILES_NAMES, FOLDER_NAME):
  print("reading file {}".format(filename))
  try:
    data = pd.read_csv(filename, index_col = None, header=0, engine = 'python',encoding = "utf-8")
    data = remove_other_langs(data)
    data = remove_other_dates(data, date_to_analyse, TEMP_FOLDER, FILES_NAMES, FOLDER_NAME)
    data = create_rt_column(data)
    return data
  except:
        return pd.DataFrame()

In [51]:
# Parse all files of the same date
def parse_all_files(TEMP_FOLDER, FILES_NAMES, date_to_analyse, FOLDER_NAME):
  print(FOLDER_NAME, "in parse all")
  files = os.listdir(os.path.join(TEMP_FOLDER, FOLDER_NAME))
  #print(files)
  new_data = []
  for file in files:
    if file.startswith(FILES_NAMES):
      parsed_data = parse_data_from_file(os.path.join(TEMP_FOLDER, FOLDER_NAME, file), date_to_analyse, TEMP_FOLDER, FILES_NAMES, FOLDER_NAME)
      new_data.append(parsed_data)
  return pd.concat(new_data, axis = 0)

In [15]:
def get_text_or_extended_text(data, added_ids):
  data["FINAL_TEXT"] = np.where(data['extended_tweet_full_text'].notnull(), data["extended_tweet_full_text"], data["text"])
  id = data["id"].to_dict()
  added_ids.update(id)
  return data["FINAL_TEXT"]

def get_quoted_text(data, added_ids):
  data["FINAL_TEXT"] = np.where(data['QT_full_text'].notnull(), data["QT_full_text"], data["QT_text"])
  added_ids.update(data["id"].to_dict())
  added_ids.update(data["QT_id"].to_dict())
  return data["FINAL_TEXT"]

def get_text_or_full_text_rt(data, added_ids):
  data["FINAL_TEXT"] = np.where(data['RT_full_text'].notnull(), data["RT_full_text"], data["RT_text"])
  added_ids.update(data["id"].to_dict())
  added_ids.update(data["RT_id"].to_dict())
  return data["FINAL_TEXT"]

def get_quote_rt_full(data, added_ids):
  data["FINAL_TEXT"] = get_text_or_full_text_rt(data, added_ids) + get_quoted_text(data, added_ids)
  return data["FINAL_TEXT"]



def get_quote_rt(data, is_quote, is_rt):
  print("get quoted text {} {}".format(is_quote, is_rt))
  data = data.loc[(data['is_quote_tweet'] == is_quote) & (data['RT'] == is_rt)]

  if not is_quote and not is_rt:
    added_ids.update(dict(zip(data["id"], data["is_quote_tweet"])))
    data["FINAL_TEXT"] = np.where(data['extended_tweet_full_text'].notnull(), data["extended_tweet_full_text"], data["text"])

  if is_quote and not is_rt:
    data["FINAL_TEXT"] = np.where(data["QT_id"].isin(added_ids.keys()), get_text_or_extended_text(data, added_ids), get_quoted_text(data, added_ids))

  if not is_quote and is_rt:
    data["FINAL_TEXT"] = np.where(data["RT_id"].isin(added_ids.keys()), added_ids.update(data['id'].to_dict()), get_text_or_full_text_rt(data, added_ids))
    data = data[data["FINAL_TEXT"].notna()]

  if is_quote and is_rt:
    data["FINAL_TEXT"] = np.where((data["RT_id"].isin(added_ids.keys())) & (data["QT_id"].isin(added_ids.keys())), added_ids.update(data['id'].to_dict()), None)
    data["FINAL_TEXT"] = np.where((data["RT_id"].isin(added_ids.keys())) & (~data["QT_id"].isin(added_ids.keys())), get_quoted_text(data, added_ids), None)
    data["FINAL_TEXT"] = np.where((~data["RT_id"].isin(added_ids.keys())) & (data["QT_id"].isin(added_ids.keys())), get_text_or_full_text_rt(data, added_ids), None)
    data["FINAL_TEXT"] = np.where((~data["RT_id"].isin(added_ids.keys())) & (~data["QT_id"].isin(added_ids.keys())), get_quote_rt_full(data, added_ids), None)
    data = data[data["FINAL_TEXT"].notna()]

  return data

In [37]:
def separate_df(filename, TEMP_FOLDER, FILES_NAMES, date_to_analyse, FOLDER_NAME):
    print(FOLDER_NAME)
    FOLDER_NAME = extract_zips(filename, TEMP_FOLDER, FOLDER_NAME)
    print(FOLDER_NAME)
    data = parse_all_files(TEMP_FOLDER, FILES_NAMES, date_to_analyse, FOLDER_NAME)

    non_quote_non_rt = get_quote_rt(data, False, False)
    quote_non_rt = get_quote_rt(data, True, False)
    quote_rt = get_quote_rt(data, True, True)
    non_quote_rt = get_quote_rt(data, False, True)
    
    return non_quote_non_rt, quote_non_rt, quote_rt, non_quote_rt
    

1. Casing (Upper or lower case)
2. Noise Removal (Removal of punctuation, white spaces, special characters, HTML tags)
3. Tokenization (Tweets to tokens i.e. words seprated by spaces)
4. Stopword Removal
5. Text Normalization (Stemming and Lemmatization) bold text

In [17]:
#Removing emojis
def demoji(text):
  emoji_pattern = re.compile("["
  u"\U0001F600-\U0001F64F"  # emoticons
                u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                u"\U0001F680-\U0001F6FF"  # transport & map symbols
                u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                u"\U00002702-\U000027B0"
                u"\U000024C2-\U0001F251"
                u"\U0001f926-\U0001f937"
                u'\U00010000-\U0010ffff'
                u"\u200d"
                u"\u2640-\u2642"
                u"\u2600-\u2B55"
                u"\u23cf"
                u"\u23e9"
                u"\u231a"
                u"\u3030"
                u"\ufe0f"
                        "]+", flags=re.UNICODE)
  return(emoji_pattern.sub(r'', text.decode('utf-8')))

#POSTags
def get_wordnet_pos(word):
  """Map POS tag to first character lemmatize() accepts"""
  tag = nltk.pos_tag([word])[0][1][0].upper()
  tag_dict = {"J": wordnet.ADJ, "N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV}
  return tag_dict.get(tag, wordnet.NOUN)

ps = PorterStemmer()

def apply_stemming(row):
  row_list = row["FINAL_TEXT"]
  stemmed_list = [ps.stem(word) for word in row_list]
  return (stemmed_list)

#Remove URLs, user@, punctutions
def df_cleaning(data_):
  punc = string.punctuation
  data_["FINAL_TEXT"] = data_["FINAL_TEXT"].astype(str).str.replace(r"http\S+| www\S+| https\S+| \S+\.com\S+| \S+\.com| \@[\w]+", "", regex=True)

  # ##################-------Punctutions-------##################
  data_["FINAL_TEXT"] = data_["FINAL_TEXT"].astype(str).str.translate(str.maketrans("", "", string.punctuation))
 
  # ##################-------More Cleaning-------##################
  data_["FINAL_TEXT"] = data_["FINAL_TEXT"].astype(str).str.replace("/[^a-zA-Z0-9 ]/g", "", regex=True).str.replace("\n"," ", regex=True).str.replace("—"," ", regex=True).str.strip("“").str.strip("”").str.strip("’").str.lstrip(" ").str.rstrip(" ")
  
  # ##################-------Emojis-------##################
#   data_["FINAL_TEXT"] = data_["FINAL_TEXT"].astype(str).apply(lambda x:demoji(x.encode('utf8')))
  data_["FINAL_TEXT"] = data_["FINAL_TEXT"].astype(str).apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))
  # ##################-------Tokenizing-------##################
  data_["FINAL_TEXT"] = data_["FINAL_TEXT"].astype(str)
  data_["FINAL_TEXT"] = data_["FINAL_TEXT"].apply(nltk.word_tokenize)
#   data_["FINAL_TEXT"] = data_.apply(lambda row: nltk.word_tokenize(row["FINAL_TEXT"]), axis=1)

  # ##################-------Lower characters---------##################
  data_["FINAL_TEXT"] = data_["FINAL_TEXT"].apply(lambda x: [word.lower() for word in x])

  # ##################-------Remove punctuations-------##################
  data_["FINAL_TEXT"] = data_["FINAL_TEXT"].apply(lambda x: [word for word in x if word not in punc])

  # ##################-------Removing stopwords-------##################
#   data_["FINAL_TEXT"] = data_["FINAL_TEXT"].astype(str).apply(lambda x: [word for word in x if word not in stop_words])
  
  # data_["FINAL_TEXT"] = data_["FINAL_TEXT"].astype(str).apply(lambda x: [word for word in x if word not in stop_words])
  data_["FINAL_TEXT"] = data_["FINAL_TEXT"].apply(lambda words: ' '.join(word for word in words if word not in stop_words))
 
  
  # ##################-------Stemming-------##################
  ps = PorterStemmer()
#   data_["FINAL_TEXT"] = data_["FINAL_TEXT"].apply(lambda x: [ps.stem(y) for y in x])
  data_["FINAL_TEXT"] = data_["FINAL_TEXT"].apply(lambda x: [ps.stem(y) for y in x.split(' ')])
#   data_["FINAL_TEXT"] = data_.apply(apply_stemming, axis = 1)
#   data_["FINAL_TEXT"] = data_.apply(ps.stem)

#   print(data_["FINAL_TEXT"], "from here")


  # ##################-------Lemmatizing-------##################
#   lemmatizer = WordNetLemmatizer()
#   data["FINAL_TEXT"] = data["FINAL_TEXT"].astype(str).apply(lambda x: [lemmatizer.lemmatize(y, get_wordnet_pos(y)) for y in x])
  
  # ##################-------Joining the lemmetized tokens to form string-------##################
  data_["FINAL_TEXT"] = data_["FINAL_TEXT"].apply(lambda x: ' '.join(x))
 
  # ##################-------Remove punctuations-------##################
#   data_["FINAL_TEXT"] = data_["FINAL_TEXT"].astype(str).str.translate(str.maketrans("", "", string.punctuation)).str.replace("’", " ").str.replace("“", " ").str.replace("”", " ")

#   final_df["FINAL_TEXT"] = data_["FINAL_TEXT"]
#   final_df = data_
  return data_


In [18]:
def split_dataframe(df, nums): 
    chunks = list()
    num_chunks = nums
    chunk_size = len(df) // nums
    for i in range(num_chunks):
        chunks.append(df[i*chunk_size:(i+1)*chunk_size])
    return chunks

In [19]:
def clean_data_and_save(data_,filename, year):
    start = time.time()
    num_cores = mp.cpu_count()
    pool = Pool(num_cores)
    df = split_dataframe(data_, num_cores)
    data1 = pd.concat(pool.map(df_cleaning, df))
    data = data1.copy()
    pool.close()
    pool.join()
    end = time.time()
    #directory = "output_data/{}-{}".format(date_to_analyse, year)
    
#     directory = "/media/manikya_varshney/Backup/Yale Data/output_data/{}-{}".format(date_to_analyse, year)
    directory = "output_data/{}-{}".format(date_to_analyse, year)
    
    create_directory(directory)
#     data1.to_hdf("{}/{}.h5".format(directory, filename), key='stage', mode='w')
    
    data1.to_csv("{}/{}.csv".format(directory, filename))
    return data

In [20]:
def clean_all_data_and_save(data_, year):
    non_quote_non_rt, quote_non_rt, quote_rt, non_quote_rt = data_
    non_quote_non_rt = clean_data_and_save(non_quote_non_rt, 'non_quote_non_rt', year)
    quote_non_rt = clean_data_and_save(quote_non_rt, 'quote_non_rt', year)    
    non_quote_rt = clean_data_and_save(non_quote_rt, 'non_quote_rt', year)
    quote_rt = clean_data_and_save(quote_rt, 'quote_rt', year)
    return non_quote_non_rt, quote_non_rt, quote_rt, non_quote_rt

In [21]:
def combine_all_data(data_, year):
    non_quote_non_rt, quote_non_rt, quote_rt, non_quote_rt = data_
    combined = pd.concat([non_quote_non_rt, quote_non_rt, quote_rt, non_quote_rt])
    #directory = "output_data/{}-{}".format(date_to_analyse, year)
    
#     directory = "/media/manikya_varshney/Backup/Yale Data/output_data/{}-{}".format(date_to_analyse, year)
    directory = "output_data/{}-{}".format(date_to_analyse, year)
    
    create_directory(directory)
#     combined.to_hdf('{}/combined_{}.csv'.format(directory, date_to_analyse), index = False, key='stage', mode='w')    
    combined.to_csv('{}/combined_{}.csv'.format(directory, date_to_analyse), index = False)

In [29]:
def final_clean(FILENAME, TEMP_FOLDER, FILES_NAMES, date_to_analyse, FOLDER_NAME, year):
    
    all_data = separate_df(FILENAME, TEMP_FOLDER, FILES_NAMES, date_to_analyse, FOLDER_NAME)
    non_quote_non_rt, quote_non_rt, quote_rt, non_quote_rt = clean_all_data_and_save(all_data, year)
    combine_all_data((non_quote_non_rt, quote_non_rt, quote_rt, non_quote_rt), year)

In [45]:
# Constants
FILENAME = "h01-20201001-20201008.zip"

# FILENAME = "/media/manikya_varshney/Backup/Yale Data/h01-20200818-20200824.zip"

TEMP_FOLDER = "tmp"

# TEMP_FOLDER = "/media/manikya_varshney/Backup/Yale Data/tmp"

FOLDER_NAME = get_folder_name_from_zip(FILENAME)

FILES_NAMES = create_filename(FILENAME, TEMP_FOLDER)
date_to_analyse, year = strip_date(FILES_NAMES)

In [53]:
start_date, end_date = get_date_range(FILENAME)

start = time.time()

# if os.path.isdir(os.path.join(TEMP_FOLDER, FOLDER_NAME)):
#     pass
# else:
#     FOLDER_NAME = ""

for single_date in daterange(start_date, end_date):
    file_name_start = 'h01-{}'.format(single_date.strftime("%Y%m%d"))
    date_to_analyse, year = strip_date(file_name_start)
    final_clean(FILENAME, TEMP_FOLDER, file_name_start, date_to_analyse, FOLDER_NAME, year)
    
    #print(date_to_analyse, year)
    #print(single_date.strftime("%Y-%m-%d"))
    
end = time.time()
print(end-start)

h01-20201001-20201008

 in parse all
reading file tmp/h01-20201001-173324.csv
reading file tmp/h01-20201001-090915.csv
reading file tmp/h01-20201001-073456.csv
reading file tmp/h01-20201001-215958.csv
20201002
reading file tmp/h01-20201001-111205.csv
reading file tmp/h01-20201001-080246.csv
reading file tmp/h01-20201001-095059.csv
reading file tmp/h01-20201001-093158.csv
reading file tmp/h01-20201001-113955.csv
reading file tmp/h01-20201001-175346.csv
reading file tmp/h01-20201001-145520.csv
reading file tmp/h01-20201001-160415.csv
reading file tmp/h01-20201001-012054.csv
reading file tmp/h01-20201001-103603.csv
reading file tmp/h01-20201001-153034.csv
reading file tmp/h01-20201001-105931.csv
reading file tmp/h01-20201001-162028.csv
reading file tmp/h01-20201001-203308.csv
20201002
reading file tmp/h01-20201001-225456.csv
reading file tmp/h01-20201001-194740.csv
reading file tmp/h01-20201001-090136.csv
reading file tmp/h01-20201001-082240.csv
reading file tmp/h01-20201001-020150.csv
re

reading file tmp/h01-20201001-152244.csv
reading file tmp/h01-20201001-234347.csv
reading file tmp/h01-20201001-072341.csv
get quoted text False False
get quoted text True False
get quoted text True True
get quoted text False True
h01-20201001-20201008

 in parse all
reading file tmp/h01-20201002-082516.csv
reading file tmp/h01-20201002-012105.csv
reading file tmp/h01-20201002-025756.csv
reading file tmp/h01-20201002-051216.csv
reading file tmp/h01-20201002-181553.csv
reading file tmp/h01-20201002-202806.csv
20201003
reading file tmp/h01-20201002-135035.csv
reading file tmp/h01-20201002-010316.csv
reading file tmp/h01-20201002-093553.csv
reading file tmp/h01-20201002-234740.csv
20201003
reading file tmp/h01-20201002-072949.csv
reading file tmp/h01-20201002-031131.csv
reading file tmp/h01-20201002-131130.csv
reading file tmp/h01-20201002-131656.csv
reading file tmp/h01-20201002-190932.csv
reading file tmp/h01-20201002-184012.csv
reading file tmp/h01-20201002-084748.csv
reading file tmp/

reading file tmp/h01-20201002-015229.csv
reading file tmp/h01-20201002-210458.csv
20201003
reading file tmp/h01-20201002-121403.csv
reading file tmp/h01-20201002-162249.csv
reading file tmp/h01-20201002-233900.csv
20201003
reading file tmp/h01-20201002-155426.csv
reading file tmp/h01-20201002-194246.csv
reading file tmp/h01-20201002-212931.csv
20201003
reading file tmp/h01-20201002-143913.csv
reading file tmp/h01-20201002-231332.csv
20201003
reading file tmp/h01-20201002-190222.csv
reading file tmp/h01-20201002-090418.csv
reading file tmp/h01-20201002-001301.csv
reading file tmp/h01-20201002-145219.csv
reading file tmp/h01-20201002-081930.csv
reading file tmp/h01-20201002-122731.csv
reading file tmp/h01-20201002-070038.csv
reading file tmp/h01-20201002-132223.csv
reading file tmp/h01-20201002-151237.csv
reading file tmp/h01-20201002-010531.csv
reading file tmp/h01-20201002-182053.csv
reading file tmp/h01-20201002-215354.csv
20201003
reading file tmp/h01-20201002-132753.csv
reading file

20201004
reading file tmp/h01-20201003-170204.csv
reading file tmp/h01-20201003-134446.csv
reading file tmp/h01-20201003-113953.csv
reading file tmp/h01-20201003-172611.csv
reading file tmp/h01-20201003-112913.csv
reading file tmp/h01-20201003-104624.csv
reading file tmp/h01-20201003-195101.csv
20201004
reading file tmp/h01-20201003-111215.csv
reading file tmp/h01-20201003-141146.csv
reading file tmp/h01-20201003-094206.csv
reading file tmp/h01-20201003-090909.csv
reading file tmp/h01-20201003-214521.csv
20201004
reading file tmp/h01-20201003-140321.csv
reading file tmp/h01-20201003-143600.csv
reading file tmp/h01-20201003-132953.csv
reading file tmp/h01-20201003-172344.csv
reading file tmp/h01-20201003-093810.csv
reading file tmp/h01-20201003-121052.csv
reading file tmp/h01-20201003-232928.csv
20201004
reading file tmp/h01-20201003-105126.csv
reading file tmp/h01-20201003-224150.csv
20201004
reading file tmp/h01-20201003-101354.csv
reading file tmp/h01-20201003-222904.csv
20201004
rea

reading file tmp/h01-20201004-150930.csv
reading file tmp/h01-20201004-002938.csv
reading file tmp/h01-20201004-131921.csv
reading file tmp/h01-20201004-031712.csv
reading file tmp/h01-20201004-061643.csv
reading file tmp/h01-20201004-130431.csv
reading file tmp/h01-20201004-000257.csv
reading file tmp/h01-20201004-093403.csv
reading file tmp/h01-20201004-153155.csv
reading file tmp/h01-20201004-103010.csv
reading file tmp/h01-20201004-155915.csv
reading file tmp/h01-20201004-235917.csv
20201005
reading file tmp/h01-20201004-124849.csv
reading file tmp/h01-20201004-193615.csv
reading file tmp/h01-20201004-124054.csv
reading file tmp/h01-20201004-171733.csv
reading file tmp/h01-20201004-001029.csv
reading file tmp/h01-20201004-170605.csv
reading file tmp/h01-20201004-131301.csv
reading file tmp/h01-20201004-120230.csv
reading file tmp/h01-20201004-021648.csv
reading file tmp/h01-20201004-100043.csv
reading file tmp/h01-20201004-124452.csv
reading file tmp/h01-20201004-071446.csv
reading

20201005
reading file tmp/h01-20201004-194942.csv
reading file tmp/h01-20201004-154435.csv
reading file tmp/h01-20201004-232052.csv
20201005
reading file tmp/h01-20201004-011141.csv
reading file tmp/h01-20201004-011800.csv
reading file tmp/h01-20201004-175404.csv
reading file tmp/h01-20201004-152520.csv
reading file tmp/h01-20201004-104000.csv
reading file tmp/h01-20201004-080300.csv
reading file tmp/h01-20201004-153819.csv
reading file tmp/h01-20201004-111342.csv
reading file tmp/h01-20201004-112105.csv
reading file tmp/h01-20201004-203503.csv
20201005
reading file tmp/h01-20201004-162145.csv
reading file tmp/h01-20201004-070555.csv
reading file tmp/h01-20201004-105121.csv
reading file tmp/h01-20201004-062958.csv
reading file tmp/h01-20201004-210635.csv
20201005
reading file tmp/h01-20201004-175029.csv
reading file tmp/h01-20201004-162555.csv
reading file tmp/h01-20201004-063854.csv
reading file tmp/h01-20201004-193033.csv
reading file tmp/h01-20201004-033246.csv
reading file tmp/h01-

20201006
reading file tmp/h01-20201005-081235.csv
reading file tmp/h01-20201005-030829.csv
reading file tmp/h01-20201005-224628.csv
20201006
reading file tmp/h01-20201005-155900.csv
reading file tmp/h01-20201005-114948.csv
reading file tmp/h01-20201005-190123.csv
reading file tmp/h01-20201005-150307.csv
reading file tmp/h01-20201005-014545.csv
reading file tmp/h01-20201005-090731.csv
reading file tmp/h01-20201005-024726.csv
reading file tmp/h01-20201005-175555.csv
reading file tmp/h01-20201005-114154.csv
reading file tmp/h01-20201005-085329.csv
reading file tmp/h01-20201005-170834.csv
reading file tmp/h01-20201005-193359.csv
reading file tmp/h01-20201005-093359.csv
reading file tmp/h01-20201005-225709.csv
reading file tmp/h01-20201005-072831.csv
reading file tmp/h01-20201005-172825.csv
reading file tmp/h01-20201005-082056.csv
reading file tmp/h01-20201005-141745.csv
reading file tmp/h01-20201005-115300.csv
reading file tmp/h01-20201005-201454.csv
20201006
reading file tmp/h01-20201005-

20201007
reading file tmp/h01-20201006-093933.csv
reading file tmp/h01-20201006-062841.csv
reading file tmp/h01-20201006-024633.csv
reading file tmp/h01-20201006-081728.csv
reading file tmp/h01-20201006-051904.csv
reading file tmp/h01-20201006-045057.csv
reading file tmp/h01-20201006-020354.csv
reading file tmp/h01-20201006-085059.csv
reading file tmp/h01-20201006-084421.csv
reading file tmp/h01-20201006-010240.csv
reading file tmp/h01-20201006-181559.csv
reading file tmp/h01-20201006-092519.csv
reading file tmp/h01-20201006-073320.csv
reading file tmp/h01-20201006-233757.csv
20201007
reading file tmp/h01-20201006-085312.csv
reading file tmp/h01-20201006-004340.csv
reading file tmp/h01-20201006-165806.csv
reading file tmp/h01-20201006-081943.csv
reading file tmp/h01-20201006-084634.csv
reading file tmp/h01-20201006-004034.csv
reading file tmp/h01-20201006-042713.csv
reading file tmp/h01-20201006-013004.csv
reading file tmp/h01-20201006-042115.csv
reading file tmp/h01-20201006-180527.cs

reading file tmp/h01-20201007-003609.csv
reading file tmp/h01-20201007-084000.csv
reading file tmp/h01-20201007-191431.csv
reading file tmp/h01-20201007-230253.csv
20201008
reading file tmp/h01-20201007-210632.csv
20201008
reading file tmp/h01-20201007-012923.csv
reading file tmp/h01-20201007-073107.csv
reading file tmp/h01-20201007-162032.csv
reading file tmp/h01-20201007-212223.csv
20201008
reading file tmp/h01-20201007-165239.csv
reading file tmp/h01-20201007-051942.csv
reading file tmp/h01-20201007-010922.csv
reading file tmp/h01-20201007-090139.csv
reading file tmp/h01-20201007-141204.csv
reading file tmp/h01-20201007-091555.csv
reading file tmp/h01-20201007-224155.csv
20201008
reading file tmp/h01-20201007-091959.csv
reading file tmp/h01-20201007-143348.csv
reading file tmp/h01-20201007-015046.csv
reading file tmp/h01-20201007-113345.csv
reading file tmp/h01-20201007-234018.csv
20201008
reading file tmp/h01-20201007-133724.csv
reading file tmp/h01-20201007-073843.csv
reading file

reading file tmp/h01-20201007-041537.csv
reading file tmp/h01-20201007-211151.csv
20201008
reading file tmp/h01-20201007-160710.csv
reading file tmp/h01-20201007-161354.csv
reading file tmp/h01-20201007-181510.csv
reading file tmp/h01-20201007-113728.csv
reading file tmp/h01-20201007-102341.csv
reading file tmp/h01-20201007-134142.csv
reading file tmp/h01-20201007-194525.csv
reading file tmp/h01-20201007-000608.csv
reading file tmp/h01-20201007-122318.csv
reading file tmp/h01-20201007-162714.csv
reading file tmp/h01-20201007-005943.csv
reading file tmp/h01-20201007-165709.csv
reading file tmp/h01-20201007-124732.csv
reading file tmp/h01-20201007-123511.csv
reading file tmp/h01-20201007-132420.csv
reading file tmp/h01-20201007-125940.csv
reading file tmp/h01-20201007-104030.csv
reading file tmp/h01-20201007-042717.csv
reading file tmp/h01-20201007-154149.csv
reading file tmp/h01-20201007-013438.csv
reading file tmp/h01-20201007-110858.csv
reading file tmp/h01-20201007-101216.csv
reading

In [54]:
shutil.rmtree(TEMP_FOLDER)

In [ ]:
'''
################ BEFORE USE ################
1. In Block [22] :- Change FILENAME and TEMP_FOLDER accordingly
2. In Block [9] :- Change directory accordingly
3. In Block [18] :- Change directory accordingly
4. In Block [20] :- Change directory accordingly

################ USAGE ################
1. In Block [22] :- Pass the location of zip file to be cleaned in FILENAME
2. In Block [23] :- Pass the name of zip file to be cleaned in get_date_range function
'''

In [50]:
def test(FOLDER_NAME):
    print(os.path.isdir(os.path.join(TEMP_FOLDER, FOLDER_NAME)))
    if os.path.isdir(os.path.join(TEMP_FOLDER, FOLDER_NAME)):
        pass
    else:
        FOLDER_NAME = ""
    return FOLDER_NAME
print(test(FOLDER_NAME))

False

